#### Ex 1: is_function
Scrivere una funzione `is_function` che prenda in input una relazione `r` (cioè una lista di coppie `(x, y)` di interi) e restituisca `true` se `r` è una funzione parziale, `false` altrimenti.

Per esempio, se `r1 = [(1,1); (2,2); (3,3); (4,4)]` e `r2 = [(1,2); (2,4); (3,6); (1,0) ]`, `is_function r1` dovrà restituire `true`, `is_function r2` dovrà resituire `false`

In [2]:
(* Per verificare che list sia una funzione, dobbiamo verirficare che non assegni 
due otuput diversi allo stesso input. Cioè, all'interno di list non devono esserci due elementi
(x1, y1) e (x2, y2) tali che x1=x2 ma y1 <> y2. Dobbiamo perciò controllare solo le coppie
che condividono la stessa x.
*)
let is_function list = 
    (*per avere tutte le coppie che condividono lo stessa x affiancate, possiamo
      ordinare la lista in base al primo elemento. La funzione List.sort si aspetta in input
      una funzione binaria f, tale che se a < b, f a b sia minore di zero, se a > b allora 
      f a b sia maggiore di zero etc.
      Visto che noi vogliamo che la coppia (x1, y1) preceda la coppia (x2, y2) solo in base
      al primo elemento, possiamo scrivere una funzione f che si dimentichi delle y e confronti
      solo le x.*)
    let sorted = List.sort (fun (x1, _) (x2, _) -> compare x1 x2) list in
    (*per controllare che due elementi (adiacenti) non abbiano la stessa x
      ma y diverse, usiamo il pattern matching per catturare i primi 2 elementi
      in testa alla lista, non solo il primo elemento*)
    let rec aux list = match list with
        |[] -> true
        |[(x, y)] -> true
        |(x1, y1)::(x2, y2)::t -> 
                    (*mettendo y1 = y2 in and con il risultato della chiamata ricorsiva,
                    la funzione viene chiamata solo se y1=y2 è verificato.
                    Cioè è del tutto equivalente a scrivere
                    if x1 = x2 then (if y1 = y2 then aux .. else false) else ..*)
                    if x1 = x2 then (y1 = y2) && aux ((x2, y2)::t)
                                    else aux ((x2, y2)::t)
    in
    aux sorted;;
    
is_function [(1,1); (2,2); (3,3); (4,4)];;
is_function [(1,2); (2,4); (3,6); (1,0)];;

val is_function : ('a * 'b) list -> bool = <fun>


- : bool = true


- : bool = false


#### Ex 2: Quicksort
Scrivere una funzione `quicksort` che usi l'algoritmo quicksort per ordinare gli elementi di una lista.

Hint: L'operazione `partition pivot list` del quicksort, nel paradigma funzionale, consiste nel costruire due nuove liste, una con tutti gli elementi di `list` minori o uguali a `pivot`, l'altra con tutti gli elementi maggiori


In [6]:
(*L'algoritmo che vogliamo implementare è:
    Se la lista è di 0 o 1 elemento, allora è già ordinata
    Se la lista è di più di 1 elemento, allora 
      1: prendiamo il primo elemento come pivot e dividiamo il resto 
        della lista in due sottoliste, left e right. left e right 
        devono contenere, rispettivamente, tutti gli elementi <= 
        del pivot, o tutti gli elementi maggiori.
      2: Ordiniamo ricorsivamente  left e right.
      3: Restituiamo left (ordinato) concatenato col pivot concatenato con right (ordinato)
*)
let rec quicksort list = 
    (*la funztione partion divide list in due sottoliste.
      Esegue uan fold su list, e come accumulatore usa una coppia di liste, left e right,
      inizialmente entrambe vuote. Ogni elemento di list che incontra viene aggiungo a left o right,
      e alla fine l'accumulatore (cioè (left, right)) conterrà tutti gli elementi di list*)
    let partition list pivot = 
        List.fold_left (
            fun (left, right) el -> if el <= pivot 
                                    then (el::left, right) 
                                    else (left, el::right)
        ) ([], []) list
    in
    match list with
    |[] -> []
    |[x] -> [x]
    |h::t ->
            (*scegliamo h come pivot, e facciamo la partition di t (senza h!)*)
            let left, right = partition t h in
                quicksort left @ [h] @ quicksort right;;

(*Al posto di implementare a mano la funzione partition, possiamo usare la funzione 
    di ordine superiore List.partition. Quest'ultima accetta in input un predicato e una lista, 
    e restituisce una coppia di liste: la prima contiene tutti gli elementi che soddisfano il predicato,
    la seconda tutti quelli che non lo soddisfano. 
    (Per predicato si intende uan funzione di un solo argomento che restituisca true o false)
    *)
let rec quicksort2 list =
 match list with
    |[] -> []
    |[x] -> [x]
    |h::t -> let left, right = List.partition (fun el -> el <= h) t in
                quicksort left @ [h] @ quicksort right;;

quicksort2 [2;4;5;3;6;4;11;3;];;

val quicksort : 'a list -> 'a list = <fun>


val quicksort2 : 'a list -> 'a list = <fun>


- : int list = [2; 3; 3; 4; 4; 5; 6; 11]


#### Ex 3: Leaves
Data la definizione di albero binario riportata sotto, scrivere una funzione `leaves` che, preso in input un albero, restituisca una lista con tutte le sue foglie.

In [9]:
type 'a binary_tree =
    | Empty
    | Node of 'a * 'a binary_tree * 'a binary_tree;;
    
let rec leaves tree = match tree with
    | Empty -> [] (*se l'albero è vuoto, non ha foglie*)
    | Node(x, Empty, Empty) -> [x] (*se c'è una sola foglia, restituiamo la lista con una sola foglia*)
    | Node(_, left, right) -> leaves left @ leaves right;;
            (*se almeno uno fra left o right sono non vuoti, allora restituiamo 
            tutte le foglie del sottoalbero di sinitra più tutte le foglie del sottoalbero di destra*)

(*La soluzione precedente ricalca esattamente una definizione "insiemistica", 
  dove [] è l'insieme vuoto e @ è l'unione. L'operatore @ però non è computazionalmente 
  efficiente, perchè richiede tempo lineare per concatenare due liste.
  Una soluzione più efficiente è quella di implementare una "fold" dell'albero,
  ovvero una operazione che percorre tutti i nodi dell'albero e costruisce man mano 
  la soluzione all'interno di un parametro accumulatore. 
  In questo caso l'accumulatore sarà una lista dentro la quale conserveremo tutte le foglie
  incontrate, ma lo stesso identico algoritmo può essere usato anche per calcolare
  la somma di tutti gli elementi, il minimo di tutti gli elementi, etc ..,
  come si fa per la fold_left o fold_right.*)
let leaves2 tree = 
    let rec aux tree acc = 
        match tree with
        |Empty -> acc 
        |Node(x, Empty, Empty) -> x::acc
        |Node(_, left, right) -> 
            (*prima raccolgo dentro leaves_left tutte le foglie
            del sottoalbero sinistro, poi ad esse aggiungo
            tutte le foglie del sottoalbero destro*)
            let leaves_left = aux left acc in
            let leaves = aux right leaves_left in
            leaves
    in
    aux tree [];; (*inizialmente l'accumulatore è vuoto*)

type 'a binary_tree = Empty | Node of 'a * 'a binary_tree * 'a binary_tree


val leaves : 'a binary_tree -> 'a list = <fun>


val leaves2 : 'a binary_tree -> 'a list = <fun>
